In [2]:
import yfinance as yf
import numpy as np 
import pandas as pd
import requests
import json
import copy

In [3]:
api_url ="https://api.usaspending.gov/api/v2/search/spending_by_award"
headers = {
    'Content-Type': 'application/json'
}

In [16]:
def setPayload(page):
    payload = {
    "subawards": False,
    "limit": 100,
    "page": page, 
    "filters": {
        "time_period": [
            {"start_date": "2007-10-01", "end_date": "2008-09-30"},
            {"start_date": "2008-10-01", "end_date": "2009-09-30"},
            {"start_date": "2009-10-01", "end_date": "2010-09-30"},
            {"start_date": "2010-10-01", "end_date": "2011-09-30"},
            {"start_date": "2011-10-01", "end_date": "2012-09-30"},
            {"start_date": "2012-10-01", "end_date": "2013-09-30"},
            {"start_date": "2013-10-01", "end_date": "2014-09-30"},
            {"start_date": "2014-10-01", "end_date": "2015-09-30"},
            {"start_date": "2015-10-01", "end_date": "2016-09-30"},
            {"start_date": "2016-10-01", "end_date": "2017-09-30"},
            {"start_date": "2017-10-01", "end_date": "2018-09-30"},
            {"start_date": "2018-10-01", "end_date": "2019-09-30"},
            {"start_date": "2019-10-01", "end_date": "2020-09-30"},
            {"start_date": "2020-10-01", "end_date": "2021-09-30"},
            {"start_date": "2021-10-01", "end_date": "2022-09-30"},
            {"start_date": "2022-10-01", "end_date": "2023-09-30"},
            {"start_date": "2023-10-01", "end_date": "2024-09-30"}, 
            {"start_date": "2024-10-01", "end_date": "2025-09-30"}
        ],
        "award_amounts": [{"lower_bound":"25000000"}],
        "award_type_codes": ["A", "B", "C", "D"],
        "naics_codes": [
            "11", "21", "22", "23", "31", "32", "33", "4233", "4235", "4238", "4246", "4247",
            "5111", "5112", "5121", "5122", "5151", "5152", "5161", "5171", "5172", "5173", "5174",
            "5175", "5179", "5181", "5182", "5191", "52", "54"
        ],
        "recipient_type_names": [
            "business", "corporate_entity_not_tax_exempt", "other_than_small_business",
            "subchapter_s_corporation", "partnership_or_limited_liability_partnership", 
            "limited_liability_corporation", "sole_proprietorship", "corporate_entity_tax_exempt", 
            "manufacturer_of_goods"
        ]
    },
    "fields": [
        "Award ID", "Recipient Name", "Award Amount", "Total Outlays", "Description", 
        "Contract Award Type", "def_codes", "COVID-19 Obligations", "COVID-19 Outlays", 
        "Infrastructure Obligations", "Infrastructure Outlays", "Awarding Agency", 
        "Awarding Sub Agency", "Start Date", "End Date", "recipient_id", "prime_award_recipient_id"
    ],
    "order": "desc",
    "sort": "Award Amount"
    }
    return payload

In [ ]:
df_contracts = {}

for i in range(100):
    file = f'contracts_data.csv'

    first_page = True
    j =380
    while True:
        payload = setPayload(j)  # Generate payload for current page
        response = requests.post(api_url, headers=headers, json=payload)
        print(f"Fetching page {j}...")

        if response.status_code == 200:
            results = response.json().get('results', [])
            if not results:  # Stop if no results
                print(f"No more data on page {j}. Stopping.")
                break

            # Convert to DataFrame and save
            df = pd.DataFrame(results, columns=payload["fields"])
            df.to_csv(file, index=False, mode='a', header=(j == 1))  
            j += 1  # Increment page

        else:
            print(f"Error: {response.status_code}, {response.text}")
            break


In [ ]:
tickers_url = "https://api.datajockey.io/v0/tickers"
query = {
    "apikey": "941a7b86a31338a1b6ccfd321d9d5bbf11abf594fba78ad2fab3"
}

response = requests.get(tickers_url, params=query)
if response.status_code == 200: 
    data = response.json()
    file = "data/tickers_data.csv"
    df = pd.DataFrame(data)
    df.to_csv(file, index=False)

In [ ]:
df = pd.read_csv("data/tickers_data.csv")
df["cik_s"] = df["cik"].apply(lambda x: str(x).zfill(10))
df = pd.DataFrame(df)
df.to_csv("data/tickers_data.csv", index=False)